In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('ml').getOrCreate()
df = spark.read.csv('data/fake_customers.csv', header=True, inferSchema=True)

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
training = spark.read.format('libsvm').load('data/sample_linear_regression_data.txt')

In [5]:
training.show() # label must be a numerical value either for linear regression value or number associated grouping

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [6]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [7]:
lr_model = lr.fit(training)

In [9]:
lr_model.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [10]:
training_summary = lr_model.summary

In [14]:
training_summary.r2

0.027839179518600154

In [13]:
training_summary.rootMeanSquaredError

10.16309157133015

In [15]:
all_data = spark.read.format('libsvm').load('data/sample_linear_regression_data.txt')

In [16]:
# randomly split the total data and put 70% in training_data and 30% in testing_data
(training_data, testing_data) = all_data.randomSplit([0.7, 0.3])

In [18]:
training_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                339|
|   mean| 0.6962854358702505|
| stddev| 10.607837205152284|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [19]:
testing_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                162|
|   mean|-0.6625892767921341|
| stddev|  9.650119749199396|
|    min|-26.805483428483072|
|    max|  22.31738046492344|
+-------+-------------------+



In [20]:
correct_model = lr.fit(training_data)

In [21]:
test_results = correct_model.evaluate(testing_data)

In [22]:
test_results.rootMeanSquaredError

9.88316487329731

In [23]:
unlabeled_data = testing_data.select('features')

In [24]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [25]:
predictions = correct_model.transform(unlabeled_data)

In [26]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|   3.760264106215848|
|(10,[0,1,2,3,4,5,...| -3.1166313433529274|
|(10,[0,1,2,3,4,5,...| 0.09732333645030566|
|(10,[0,1,2,3,4,5,...|  2.3796402871507008|
|(10,[0,1,2,3,4,5,...|  1.9645694001712413|
|(10,[0,1,2,3,4,5,...|   2.789344180743603|
|(10,[0,1,2,3,4,5,...|   2.921509289619088|
|(10,[0,1,2,3,4,5,...|  -0.769508045990509|
|(10,[0,1,2,3,4,5,...|  0.0937054633111305|
|(10,[0,1,2,3,4,5,...|  3.6611105389522542|
|(10,[0,1,2,3,4,5,...| 0.45481258901909993|
|(10,[0,1,2,3,4,5,...| -1.4375168673900491|
|(10,[0,1,2,3,4,5,...|  2.3662822988668015|
|(10,[0,1,2,3,4,5,...|  -2.484685629252359|
|(10,[0,1,2,3,4,5,...|-0.47297722829678723|
|(10,[0,1,2,3,4,5,...|    3.15012769521719|
|(10,[0,1,2,3,4,5,...|0.011247225497858504|
|(10,[0,1,2,3,4,5,...|  1.4555106366963382|
|(10,[0,1,2,3,4,5,...|  3.8461432275949488|
|(10,[0,1,2,3,4,5,...| -1.935962